In [1]:
import pandas as pd
import numpy as np
import os
directory = 'modelos/st'
if not os.path.exists(directory):
    os.makedirs(directory)
# Guardo modelo como un pickle file 
from joblib import dump 

In [2]:
auctions = pd.read_csv('../datos/auctions.csv', low_memory=False)

# Preparo el df y divido ventanas

In [3]:
auctions['date'] = pd.to_datetime(auctions['date'])

In [4]:
w1 = auctions[auctions['date'].dt.day < 21]
w2 = auctions[(auctions['date'].dt.day > 18) & (auctions['date'].dt.day < 22)]
w3 = auctions[(auctions['date'].dt.day > 19) & (auctions['date'].dt.day < 23)]
w4 = auctions[(auctions['date'].dt.day > 20) & (auctions['date'].dt.day < 24)]
w5 = auctions[(auctions['date'].dt.day > 21) & (auctions['date'].dt.day < 25)]
w6 = auctions[(auctions['date'].dt.day > 22) & (auctions['date'].dt.day < 26)]
w7 = auctions[(auctions['date'].dt.day > 23) & (auctions['date'].dt.day < 27)]

In [5]:
windowStart = pd.to_datetime("2019-04-18 00:00:00.000000")
w1['timeFromWindowStart'] = (w1['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-19 00:00:00.000000")
w2['timeFromWindowStart'] = (w2['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-20 00:00:00.000000")
w3['timeFromWindowStart'] = (w3['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-21 00:00:00.000000")
w4['timeFromWindowStart'] = (w4['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-22 00:00:00.000000")
w5['timeFromWindowStart'] = (w5['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-23 00:00:00.000000")
w6['timeFromWindowStart'] = (w6['date'] - windowStart)/np.timedelta64(1,'s')
windowStart = pd.to_datetime("2019-04-23 00:00:00.000000")
w7['timeFromWindowStart'] = (w7['date'] - windowStart)/np.timedelta64(1,'s')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org

In [6]:
w1 = w1.sort_values(by=['device_id', 'date'])
w1 = w1.drop_duplicates('device_id', keep = 'first')
w2 = w2.sort_values(by=['device_id', 'date'])
w2 = w2.drop_duplicates('device_id', keep = 'first')
w3 = w3.sort_values(by=['device_id', 'date'])
w3 = w3.drop_duplicates('device_id', keep = 'first')
w4 = w4.sort_values(by=['device_id', 'date'])
w4 = w4.drop_duplicates('device_id', keep = 'first')
w5 = w5.sort_values(by=['device_id', 'date'])
w5 = w5.drop_duplicates('device_id', keep = 'first')
w6 = w6.sort_values(by=['device_id', 'date'])
w6 = w6.drop_duplicates('device_id', keep = 'first')
w7 = w7.sort_values(by=['device_id', 'date'])
w7 = w7.drop_duplicates('device_id', keep = 'first')

In [7]:
w1.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w2.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w3.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w4.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w5.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w6.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']
w7.columns = ['date', 'device_id', 'ref_type_id', 'source_id', 'st']

In [8]:
w1 = w1.drop(columns = ['date'])
w2 = w2.drop(columns = ['date'])
w3 = w3.drop(columns = ['date'])
w4 = w4.drop(columns = ['date'])
w5 = w5.drop(columns = ['date'])
w6 = w6.drop(columns = ['date'])
w7 = w7.drop(columns = ['date'])

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
36601349,40621409780134,1,0,226857.222979,82963.592845,75012.201701
33917684,168103949904656,1,3,173044.526198,82963.592845,81005.529793
9402375,186034136943920,1,1,88.851558,82963.592845,75747.944748
13736755,295841792051458,1,1,432.827168,82963.592845,75747.944748
43932908,345999128501141,1,0,164703.549846,82963.592845,75012.201701


# Explicación

Para calcular finalmente con XGBoost, nosotros agregamos predicciones con otros métodos como features así que
vamos a usar el entrenamiento que hicimos durante el análisis exploratorio para predecir estos features y poder
agregar las columnas que se necesitan para la predicción final

## Aplico KNN y RandomForest 
Aplico KNN y RandomForest en la primer ventana y hago las predicciones de la cuarta ventana y despues le voy a agregar esas predicciones como features

In [11]:
X, y = w1.iloc[:,:-1],w1.iloc[:,-1]

In [12]:
neigh = joblib.load('modelos/st/neigh.pkl')

In [32]:
preds_KNN_w4 = neigh.predict(w4.iloc[:,:-1])

In [16]:
rf = joblib.load('modelos/st/rf.pkl')

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [33]:
preds_RF_w4 = rf.predict(w4.iloc[:,:-1])

## Entreno con w4
Ya me guarde las predicciones de w4 de RF y de KNN y las voy a agregar como features luego.
Ahora voy a entrenar con w4 y validar con w7 y luego les voy a agregar tanto a w4 como a w7 los features de las predicciones de KNN y de RF

In [37]:
preds_KNN_w7 = neigh.predict(w7.iloc[:,:-1])

In [40]:
preds_RF_w7 = rf.predict(w7.iloc[:,:-1])

In [41]:
w4["pred_KNN"] = preds_KNN_w4

In [42]:
w4["pred_RF"] = preds_RF_w4
w7["pred_KNN"] = preds_KNN_w7
w7["pred_RF"] = preds_RF_w7

In [45]:
w4.columns = ['device_id', 'ref_type_id', 'source_id', 'pred_KNN', 'pred_RF', 'st']
w7.columns = ['device_id', 'ref_type_id', 'source_id', 'pred_KNN', 'pred_RF', 'st']

In [47]:
w4.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
36601349,40621409780134,1,0,226857.222979,82963.592845,75012.201701
33917684,168103949904656,1,3,173044.526198,82963.592845,81005.529793
9402375,186034136943920,1,1,88.851558,82963.592845,75747.944748
13736755,295841792051458,1,1,432.827168,82963.592845,75747.944748
43932908,345999128501141,1,0,164703.549846,82963.592845,75012.201701


In [48]:
w7.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
39675750,69039685746313,1,7,212658.597103,89392.758783,75890.993556
1528620,345999128501141,1,2,336762.048531,89392.758783,87800.256731
20706804,360710529886978,1,1,133087.256609,89392.758783,79290.162968
35163923,365882020742330,1,8,183027.763854,89392.758783,184289.399513
20794330,416301579449694,1,1,98969.446617,89392.758783,79290.162968


## Ahora voy a entrenar w4 y validar con w7 con XBBoost con las nuevas features

In [49]:
X, y = w4.iloc[:,:-1],w4.iloc[:,-1]

In [52]:
xg_reg = joblib.load('modelos/st/xg_reg.pkl')

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.9, gamma=0, learning_rate=0.8, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [53]:
preds = xg_reg.predict(w7.iloc[:,:-1])

In [54]:
rmse = np.sqrt(mean_squared_error(w7.iloc[:,-1], preds))
print("RMSE: %f" % (rmse))

RMSE: 7362.910028


## Preparo el target_competencia

In [55]:
targets_st = pd.read_csv('target_competencia_ids.csv')
targets_st = targets_st[~targets_st['ref_hash'].str.contains('_sc')]
targets_st['ref_hash'] = targets_st['ref_hash'].map(lambda x: str(x)[:-3])

In [56]:
targets_st['ref_hash'] = targets_st['ref_hash'].astype(np.int64)
targets_st.columns = ['device_id', 'date_dif']

In [57]:
auctions_without_duplicates = auctions.drop_duplicates(subset = 'device_id')

In [58]:
targets_st = targets_st.merge(auctions_without_duplicates[['device_id', 'ref_type_id', 'source_id']], on = 'device_id', how = 'left')

In [59]:
targets_st = targets_st.drop(columns = ['date_dif'])

## Predigo con KNN y con RF target compentencia para agregárcelos como features

In [60]:
w7 = w7.drop(columns = ['pred_KNN'])
w7 = w7.drop(columns = ['pred_RF'])

In [61]:
w7.head()

,device_id,ref_type_id,source_id,st
39675750,69039685746313,1,7,75890.993556
1528620,345999128501141,1,2,87800.256731
20706804,360710529886978,1,1,79290.162968
35163923,365882020742330,1,8,184289.399513
20794330,416301579449694,1,1,79290.162968


In [62]:
X, y = w7.iloc[:,:-1],w7.iloc[:,-1]

In [65]:
preds_KNN_targets = neigh.predict(targets_st)

In [68]:
preds_RF_targets = rf.predict(targets_st)

In [69]:
w7["pred_KNN"] = preds_KNN_w7
w7["pred_RF"] = preds_RF_w7
targets_st["pred_KNN"] = preds_KNN_targets
targets_st["pred_RF"] = preds_RF_targets

In [73]:
w7.columns = ['device_id', 'ref_type_id', 'source_id', 'pred_KNN', 'pred_RF', 'st']

In [74]:
w7.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF,st
39675750,69039685746313,1,7,75890.993556,89392.758783,75890.993556
1528620,345999128501141,1,2,87800.256731,89392.758783,87800.256731
20706804,360710529886978,1,1,79290.162968,89392.758783,79290.162968
35163923,365882020742330,1,8,184289.399513,89392.758783,184289.399513
20794330,416301579449694,1,1,79290.162968,89392.758783,79290.162968


In [75]:
targets_st.head()

,device_id,ref_type_id,source_id,pred_KNN,pred_RF
0,1000169251625791246,1,1,81390.863459,79290.162968
1,1000395625957344683,1,8,81172.476531,91940.069335
2,1003027494996471685,1,0,81915.569857,83242.418682
3,1006670001679961544,7,0,80692.622639,83242.418682
4,1007573308966476713,1,1,81110.568799,79290.162968


## Ahora aplico XGBoost entrendando con w7 y prediciendo targets

In [76]:
X, y = w7.iloc[:,:-1],w7.iloc[:,-1]

In [80]:
preds = xg_reg.predict(targets_st)

In [81]:
for pred in preds: print(pred)

79631.73
79679.83
82655.695
79636.414
79631.73
82655.695
82726.28
82651.01
82651.01
79631.73
79631.73
79658.24
82652.92
82655.695
82651.01
79633.64
79649.77
79636.414
83211.8
82655.695
82669.055
82652.92
82651.01
82651.01
88354.49
79631.73
79640.195
79636.414
79631.73
79631.73
82651.01
82651.01
79640.195
82652.92
82655.695
82651.01
82655.695
82651.01
82814.195
82669.055
82655.695
82651.01
82651.01
79631.73
79649.77
79633.64
82659.48
79631.73
79636.414
79631.73
82661.39
79636.414
79649.77
79636.414
82734.75
82651.01
82651.01
82651.01
82651.01
82651.01
79631.73
79631.73
79633.64
79631.73
79636.414
82652.92
82730.06
79631.73
79636.414
79631.73
79631.73
79649.77
79679.83
79631.73
79631.73
82651.01
82826.43
82651.01
83229.84
82652.92
82669.055
82655.695
82655.695
82651.01
79631.73
79631.73
82651.01
82669.055
82655.695
82652.92
79633.64
79633.64
79631.73
79644.88
82769.695
82651.01
82655.695
82651.01
79633.64
79631.73
82651.01
79631.73
82652.92
82655.695
82652.92
82655.695
79636.414
79631.73